# Eu Horizon

In [1]:
# Artikel
# <div id="001">
# Absatz: <div id="001.001"> 

# Que nem o explanation code

In [7]:
import requests 
from bs4 import BeautifulSoup 
import unicodedata
import numpy as np
import pandas as pd

In [8]:
## Getting the law:

URL = "https://eur-lex.europa.eu/legal-content/DE/TXT/HTML/?uri=CELEX:32021R0695&from=EN#d1e1140-1-1"
r = requests.get(URL) 
soup = BeautifulSoup(r.content, 'html5lib')
law = 'eu_horizon'

c:\Python310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [9]:
## Getting all the artikels and separating Artikels with and without Absatze:

#1. getting all artikels and absatze:

ids = [tag['id'] for tag in soup.select('div[id]')]

df = pd.DataFrame(ids)
df.rename(columns={list(df)[0]:'ids'}, inplace=True)
df['artikel'] = df['ids'].str[:3]
df

#2. Looking which one of them have Absatze

df1 = df.groupby(['artikel']).count().reset_index()
df1 = df1.query("ids > 1")
ids_mit = df1['artikel']#.astype('str')

#3. Getting the final ids, so the ids and ids with absatze dont appear twice:
ids_final = set(ids) - set(ids_mit)

## Creating the table with all Artikel and Absatze:

#1. Creating empty df:
df_law = pd.DataFrame(columns = ['original_id','gesetzt', 'artikel', 'absatz', 'text'])

#2. loop for ids:

for id_entry in ids_final:
    #pegando tudo:
    table = soup.find('div', attrs = {'id':id_entry}) 
    my_str = table.text
    result = unicodedata.normalize('NFKD', my_str)   # removind \xa0
    result = " ".join(result.split())
    #result # lindo, está o absatz completo
    #agora pegando os artigos e os absatz
    #criando um df temporario com os detalhes do artigo:
    if len(id_entry) < 4:
        df_temp = pd.DataFrame({'original_id': [id_entry],
                                'gesetzt': [law],  
                                'artikel': [id_entry.split('.')[0]], 
                                'absatz' : ["NA"],
                                'text': [result]})
    else:
        df_temp = pd.DataFrame({'original_id': [id_entry],
                                'gesetzt': [law],  
                                'artikel': [id_entry.split('.')[0]], 
                                'absatz' : [id_entry.split('.')[1]],
                                'text': [result]})
    #df_temp.dtypes funciona
    df_law = pd.concat([df_law, df_temp])

df_law

,original_id,gesetzt,artikel,absatz,text
0,022.006,eu_horizon,022,006,(6) Soweit erforderlich und hinreichend begrü...
0,053.002,eu_horizon,053,002,(2) Die Rechnungsprüfungsstrategie für das P...
0,050.001,eu_horizon,050,001,(1) Die Kommission überwacht kontinuierlich d...
0,028.001,eu_horizon,028,001,(1) Die Vorschläge werden auf der Grundlage d...
0,022.004,eu_horizon,022,004,(4) Koordinierungs- und Unterstützungsmaßnahm...
...,...,...,...,...,...
0,047.002,eu_horizon,047,002,(2) Die Übergangstätigkeiten im Rahmen von P...
0,009.001,eu_horizon,009,001,(1) Die Kommission richtet den EIC als zentral...
0,024.002,eu_horizon,024,002,(2) Wenn dies zur Erreichung ihrer Ziele notwe...
0,008.005,eu_horizon,008,005,(5) Die Kommission überwacht und bewertet jed...


In [10]:
# removing anhang (letters different from L)

df_law = df_law[df_law['artikel'].str[0] != 'L']

In [6]:
# Salvando

df_law.to_csv("eu_horizon.csv")